In [12]:
import sys
sys.path.append("../")
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['STANFORD_NER_PATH'] = "../stanford-ner-2020-11-17"

In [13]:
import pandas as pd
from utils import data_cleaning
from info_extraction.ie_rules import run_ie as run_rules, rules_info_ext
from info_extraction.ie_spacy import run_ie as run_spacy, spacy_info_ext
from info_extraction.ie_stanford import run_ie as run_stanford, stanford_info_ext

In [28]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
annotated_data = [
    {
        "tweet": "What a thrilling match! Rahul's magnificent century led Punjab Kings to victory over Delhi Capitals. #PunjabKingsvsDelhiCapitals",
        "players": ["Rahul"],
        "teams": ["Punjab Kings", "Delhi Capitals"]
    },
    {
        "tweet": "Superb performance by Dhoni as he steers Chennai Super Kings to a win against Sunrisers Hyderabad. #ChennaiSuperKingsvsSunrisersHyderabad",
        "players": ["Dhoni"],
        "teams": ["Chennai Super Kings", "Sunrisers Hyderabad"]
    },
    {
        "tweet": "Incredible bowling display by Bumrah helps Mumbai Indians defend a low total against Rajasthan Royals. #MumbaiIndiansvsRajasthanRoyals",
        "players": ["Bumrah"],
        "teams": ["Mumbai Indians", "Rajasthan Royals"]
    },
    {
        "tweet": "Great partnership between Kohli and de Villiers sets up a huge total for Royal Challengers Bangalore against Kolkata Knight Riders. #RoyalChallengersBangalorevsKolkataKnightRiders",
        "players": ["Kohli", "de Villiers"],
        "teams": ["Royal Challengers Bangalore", "Kolkata Knight Riders"]
    },
    {
        "tweet": "Russell's explosive batting takes KKR to victory over RR in a nail-biting finish. #KolkataKnightRidersvsRajasthanRoyals",
        "players": ["Russell"],
        "teams": ["KKR", "RR"]
    },
    {
        "tweet": "Bravo's stunning catch secured victory for CSK against MI. What a game! #ChennaiSuperKingsvsMumbaiIndians",
        "players": ["Bravo"],
        "teams": ["CSK", "MI"]
    },
    {
        "tweet": "Kuldeep Yadav's magical spell helps Kolkata Knight Riders restrict SRH to a low score. #KolkataKnightRidersvsSunrisersHyderabad",
        "players": ["Kuldeep Yadav"],
        "teams": ["Kolkata Knight Riders", "SRH"]
    },
    {
        "tweet": "Shreyas Iyer's captaincy leads DC to a comfortable win over Rajasthan Royals. #DelhiCapitalsvsRajasthanRoyals",
        "players": ["Shreyas Iyer"],
        "teams": ["DC", "Rajasthan Royals"]
    },
    {
        "tweet": "Impressive all-round performance by Hardik Pandya guides Mumbai Indians to victory over RCB. #MumbaiIndiansvsRoyalChallengersBangalore",
        "players": ["Hardik Pandya"],
        "teams": ["Mumbai Indians", "RCB"]
    },
    {
        "tweet": "Warner's masterclass guides Sunrisers Hyderabad to a convincing win over PBKS. #SunrisersHyderabadvsPunjabKings",
        "players": ["Warner"],
        "teams": ["Sunrisers Hyderabad", "PBKS"]
    },
    {
        "tweet": "Vintage Gayle fireworks help Punjab Kings chase down a massive target against RR. #PunjabKingsvsRajasthanRoyals",
        "players": ["Gayle"],
        "teams": ["Punjab Kings", "RR"]
    },
    {
        "tweet": "Magnificent century by Stokes goes in vain as Rajasthan Royals fall short against Chennai Super Kings. #RajasthanRoyalsvsChennaiSuperKings",
        "players": ["Stokes"],
        "teams": ["Rajasthan Royals", "Chennai Super Kings"]
    },
    {
        "tweet": "Clinical performance by Rabada ensures a comfortable win for Delhi Capitals over KKR. #DelhiCapitalsvsKolkataKnightRiders",
        "players": ["Rabada"],
        "teams": ["Delhi Capitals", "KKR"]
    },
    {
        "tweet": "Dhawan's classy innings helps DC edge past MI in a last-over thriller. #DelhiCapitalsvsMumbaiIndians",
        "players": ["Dhawan"],
        "teams": ["DC", "MI"]
    },
    {
        "tweet": "Brilliant captaincy by Smith leads RR to a thrilling win over Royal Challengers Bangalore. #RajasthanRoyalsvsRoyalChallengersBangalore",
        "players": ["Smith"],
        "teams": ["RR", "Royal Challengers Bangalore"]
    },
    {
        "tweet": "Pandya's blistering knock lifts Mumbai Indians to victory over Kolkata Knight Riders. Pandya was unstoppable tonight! #MumbaiIndiansvsKolkataKnightRiders",
        "players": ["Pandya"],
        "teams": ["Mumbai Indians", "Kolkata Knight Riders"]
    },
    {
        "tweet": "Unbelievable innings by de Villiers powers RCB to a massive total against SRH. #RoyalChallengersBangalorevsSunrisersHyderabad",
        "players": ["de Villiers"],
        "teams": ["RCB", "SRH"]
    },
    {
        "tweet": "Sensational bowling by Rashid Khan helps Sunrisers Hyderabad defend a low total against CSK. #SunrisersHyderabadvsChennaiSuperKings",
        "players": ["Rashid Khan"],
        "teams": ["Sunrisers Hyderabad", "CSK"]
    },
    {
        "tweet": "Outstanding performance by Archer as Rajasthan Royals win against PBKS. #RajasthanRoyalsvsPunjabKings",
        "players": ["Archer"],
        "teams": ["Rajasthan Royals", "PBKS"]
    },
    {
        "tweet": "Masterful innings by Williamson leads SRH to victory over Delhi Capitals. #SunrisersHyderabadvsDelhiCapitals",
        "players": ["Williamson"],
        "teams": ["SRH", "Delhi Capitals"]
    },
    {
        "tweet": "Fantastic spell by Chahal helps Royal Challengers Bangalore defend a low total against KKR. #RoyalChallengersBangalorevsKolkataKnightRiders",
        "players": ["Chahal"],
        "teams": ["Royal Challengers Bangalore", "KKR"]
    },
    {
        "tweet": "Samson's brilliant century powers RR to victory over MI. What a knock! #RajasthanRoyalsvsMumbaiIndians",
        "players": ["Samson"],
        "teams": ["RR", "MI"]
    },
    {
        "tweet": "Incredible batting display by Warner and Bairstow leads Sunrisers Hyderabad to victory over Punjab Kings. #SunrisersHyderabadvsPunjabKings",
        "players": ["Warner", "Bairstow"],
        "teams": ["Sunrisers Hyderabad", "Punjab Kings"]
    },
    {
        "tweet": "Ferguson's fiery spell dismantles the batting order of RCB as Kolkata Knight Riders secure a win. #KolkataKnightRidersvsRoyalChallengersBangalore",
        "players": ["Ferguson"],
        "teams": ["Kolkata Knight Riders", "RCB"]
    },
    {
        "tweet": "Dominant performance by Mumbai Indians as they crush PBKS by a huge margin. #MumbaiIndiansvsPunjabKings",
        "players": [],
        "teams": ["Mumbai Indians", "PBKS"]
    },
    {
        "tweet": "Clinical chase by Gayle and Rahul ensures a comfortable victory for Punjab Kings over DC. #PunjabKingsvsDelhiCapitals",
        "players": ["Gayle", "Rahul"],
        "teams": ["Punjab Kings", "DC"]
    },
    {
        "tweet": "Brilliant all-round display by Pandya brothers guides MI to victory over Rajasthan Royals. #MumbaiIndiansvsRajasthanRoyals",
        "players": ["Pandya"],
        "teams": ["MI", "Rajasthan Royals"]
    },
    {
        "tweet": "Captaincy masterstroke by Kohli as Royal Challengers Bangalore clinch a last-ball thriller against SRH. #RoyalChallengersBangalorevsSunrisersHyderabad",
        "players": ["Kohli"],
        "teams": ["Royal Challengers Bangalore", "SRH"]
    },
    {
        "tweet": "Narine's quickfire innings propels KKR to a challenging total against Chennai Super Kings. #KolkataKnightRidersvsChennaiSuperKings",
        "players": ["Narine"],
        "teams": ["KKR", "Chennai Super Kings"]
    },
    {
        "tweet": "Sensational century by Iyer helps Delhi Capitals set up a huge target for RR. #DelhiCapitalsvsRajasthanRoyals",
        "players": ["Iyer"],
        "teams": ["Delhi Capitals", "RR"]
    },
    {
        "tweet": "Vintage Dhoni finishes the game in style for CSK against Mumbai Indians. #ChennaiSuperKingsvsMumbaiIndians",
        "players": ["Dhoni"],
        "teams": ["CSK", "Mumbai Indians"]
    },
    {
        "tweet": "Stunning batting display by Pandya brothers takes MI to a convincing win over Sunrisers Hyderabad. #MumbaiIndiansvsSunrisersHyderabad",
        "players": ["Pandya"],
        "teams": ["MI", "Sunrisers Hyderabad"]
    },
    {
        "tweet": "Rohit Sharma's classy century helps Mumbai Indians cruise to victory over Kolkata Knight Riders. #MumbaiIndiansvsKolkataKnightRiders",
        "players": ["Rohit Sharma"],
        "teams": ["Mumbai Indians", "Kolkata Knight Riders"]
    },
    {
        "tweet": "Magnificent spell by Archer rattles the batting lineup of RCB. #RajasthanRoyalsvsRoyalChallengersBangalore",
        "players": ["Archer"],
        "teams": ["RCB"]
    },
    {
        "tweet": "Dominant performance by Bravo as Chennai Super Kings thrash DC by a huge margin. #ChennaiSuperKingsvsDelhiCapitals",
        "players": ["Bravo"],
        "teams": ["Chennai Super Kings", "DC"]
    },
    {
        "tweet": "Exceptional bowling performance by Rashid Khan helps SRH defend a low total against PBKS. #SunrisersHyderabadvsPunjabKings",
        "players": ["Rashid Khan"],
        "teams": ["SRH", "PBKS"]
    },
    {
        "tweet": "Buttler's explosive batting guides Rajasthan Royals to victory over Royal Challengers Bangalore. #RajasthanRoyalsvsRoyalChallengersBangalore",
        "players": ["Buttler"],
        "teams": ["Rajasthan Royals", "Royal Challengers Bangalore"]
    },
    {
        "tweet": "Outstanding performance by Ferguson as KKR edge past CSK in a last-over thriller. #KolkataKnightRidersvsChennaiSuperKings",
        "players": ["Ferguson"],
        "teams": ["KKR", "CSK"]
    },
    {
        "tweet": "Clinical chase by Gayle and Mayank ensures a comfortable victory for Punjab Kings over Sunrisers Hyderabad. #PunjabKingsvsSunrisersHyderabad",
        "players": ["Gayle", "Mayank"],
        "teams": ["Punjab Kings", "Sunrisers Hyderabad"]
    },
    {
        "tweet": "Dhawan's splendid century powers Delhi Capitals to a massive total against MI. #DelhiCapitalsvsMumbaiIndians",
        "players": ["Dhawan"],
        "teams": ["Delhi Capitals", "MI"]
    },
    {
        "tweet": "Stokes' heroic innings helps RR chase down a challenging target against Kolkata Knight Riders. #RajasthanRoyalsvsKolkataKnightRiders",
        "players": ["Stokes"],
        "teams": ["RR", "Kolkata Knight Riders"]
    },
    {
        "tweet": "Bravo's all-round brilliance leads Chennai Super Kings to victory over RCB. #ChennaiSuperKingsvsRoyalChallengersBangalore",
        "players": ["Bravo"],
        "teams": ["Chennai Super Kings", "RCB"]
    },
    {
        "tweet": "Superb partnership between Pandey and Warner takes SRH to a convincing win over Rajasthan Royals. #SunrisersHyderabadvsRajasthanRoyals",
        "players": ["Pandey", "Warner"],
        "teams": ["SRH", "Rajasthan Royals"]
    },
    {
        "tweet": "Sensational bowling by Chahar helps Mumbai Indians defend a low total against PBKS. #MumbaiIndiansvsPunjabKings",
        "players": ["Chahar"],
        "teams": ["Mumbai Indians", "PBKS"]
    },
    {
        "tweet": "Clinical performance by Rabada as DC restrict Royal Challengers Bangalore to a low total. #DelhiCapitalsvsRoyalChallengersBangalore",
        "players": ["Rabada"],
        "teams": ["DC", "Royal Challengers Bangalore"]
    },
    {
        "tweet": "Rajpoot's superb spell dismantles the batting order of Sunrisers Hyderabad as Punjab Kings secure a win. #PunjabKingsvsSunrisersHyderabad",
        "players": ["Rajpoot"],
        "teams": ["Punjab Kings", "Sunrisers Hyderabad"]
    },
    {
        "tweet": "Impressive batting display by de Villiers and Padikkal sets up a huge total for RCB against MI. #RoyalChallengersBangalorevsMumbaiIndians",
        "players": ["de Villiers", "Padikkal"],
        "teams": ["RCB", "MI"]
    },
    {
        "tweet": "Watson's explosive innings helps CSK chase down a massive target against KKR. #ChennaiSuperKingsvsKolkataKnightRiders",
        "players": ["Watson"],
        "teams": ["CSK", "KKR"]
    },
    {
        "tweet": "Clinical bowling performance by Nortje helps Delhi Capitals defend a low total against RR. #DelhiCapitalsvsRajasthanRoyals",
        "players": ["Nortje"],
        "teams": ["Delhi Capitals", "RR"]
    },
    {
        "tweet": "Brilliant all-round performance by Morris guides Royal Challengers Bangalore to victory over PBKS. #RoyalChallengersBangalorevsPunjabKings",
        "players": ["Morris"],
        "teams": ["Royal Challengers Bangalore", "PBKS"]
    },
    {
        "tweet": "Masterful innings by Samson leads Rajasthan Royals to victory over Chennai Super Kings. #RajasthanRoyalsvsChennaiSuperKings",
        "players": ["Samson"],
        "teams": ["Rajasthan Royals", "Chennai Super Kings"]
    },
    {
        "tweet": "Stunning batting display by Pandya brothers helps Mumbai Indians set up a huge total against SRH. #MumbaiIndiansvsSunrisersHyderabad",
        "players": ["Pandya"],
        "teams": ["Mumbai Indians", "SRH"]
    },
    {
        "tweet": "Incredible performance by Archer as RR secure a win over Kolkata Knight Riders. #RajasthanRoyalsvsKolkataKnightRiders",
        "players": ["Archer"],
        "teams": ["RR", "Kolkata Knight Riders"]
    },
    {
        "tweet": "Sensational century by Rahul goes in vain as Punjab Kings fall short against RCB. #PunjabKingsvsRoyalChallengersBangalore",
        "players": ["Rahul"],
        "teams": ["Punjab Kings", "RCB"]
    },
    {
        "tweet": "Vintage Gayle storm helps PBKS chase down a massive target against MI. #PunjabKingsvsMumbaiIndians",
        "players": ["Gayle"],
        "teams": ["PBKS", "MI"]
    },
    {
        "tweet": "Bravo's stunning catch secures victory for CSK against DC. What a game! #ChennaiSuperKingsvsDelhiCapitals",
        "players": ["Bravo"],
        "teams": ["CSK", "DC"]
    },
    {
        "tweet": "Dominant performance by Rashid Khan as Sunrisers Hyderabad thrash KKR by a huge margin. #SunrisersHyderabadvsKolkataKnightRiders",
        "players": ["Rashid Khan"],
        "teams": ["Sunrisers Hyderabad", "KKR"]
    },
    {
        "tweet": "Clinical chase by de Villiers and Kohli ensures a comfortable victory for Royal Challengers Bangalore over Rajasthan Royals. #RoyalChallengersBangalorevsRajasthanRoyals",
        "players": ["de Villiers", "Kohli"],
        "teams": ["Royal Challengers Bangalore", "Rajasthan Royals"]
    },
    {
        "tweet": "Magnificent century by Dhawan leads Delhi Capitals to victory over SRH. #DelhiCapitalsvsSunrisersHyderabad",
        "players": ["Dhawan"],
        "teams": ["Delhi Capitals", "SRH"]
    },
    {
        "tweet": "Stunning bowling display by Bumrah as Mumbai Indians defend a low total against Chennai Super Kings. #MumbaiIndiansvsChennaiSuperKings",
        "players": ["Bumrah"],
        "teams": ["Mumbai Indians", "Chennai Super Kings"]
    },
    {
        "tweet": "Vintage Gayle fireworks help Punjab Kings chase down a challenging target against RCB. #PunjabKingsvsRoyalChallengersBangalore",
        "players": ["Gayle"],
        "teams": ["Punjab Kings", "RCB"]
    },
    {
        "tweet": "Buttler's blistering knock takes RR to a convincing win over Sunrisers Hyderabad. #RajasthanRoyalsvsSunrisersHyderabad",
        "players": ["Buttler"],
        "teams": ["RR", "Sunrisers Hyderabad"]
    },
    {
        "tweet": "Unstoppable Warner leads SRH to victory over Kolkata Knight Riders. #SunrisersHyderabadvsKolkataKnightRiders",
        "players": ["Warner"],
        "teams": ["SRH", "Kolkata Knight Riders"]
    },
    {
        "tweet": "Kohli's masterclass guides Royal Challengers Bangalore to a comfortable win over CSK. #RoyalChallengersBangalorevsChennaiSuperKings",
        "players": ["Kohli"],
        "teams": ["Royal Challengers Bangalore", "CSK"]
    },
    {
        "tweet": "Sensational performance by Archer as Rajasthan Royals clinch a last-ball thriller against MI. #RajasthanRoyalsvsMumbaiIndians",
        "players": ["Archer"],
        "teams": ["Rajasthan Royals", "MI"]
    },
    {
        "tweet": "Fantastic spell by Chahal helps RCB defend a low total against DC. #RoyalChallengersBangalorevsDelhiCapitals",
        "players": ["Chahal"],
        "teams": ["RCB", "DC"]
    },
    {
        "tweet": "Incredible century by Watson takes Chennai Super Kings to a convincing win over PBKS. #ChennaiSuperKingsvsPunjabKings",
        "players": ["Watson"],
        "teams": ["Chennai Super Kings", "PBKS"]
    },
    {
        "tweet": "Dominant performance by Morris as Royal Challengers Bangalore thrash KKR by a huge margin. #RoyalChallengersBangalorevsKolkataKnightRiders",
        "players": ["Morris"],
        "teams": ["Royal Challengers Bangalore", "KKR"]
    },
    {
        "tweet": "Stokes' brilliant all-round display helps RR edge past Delhi Capitals in a nail-biting finish. #RajasthanRoyalsvsDelhiCapitals",
        "players": ["Stokes"],
        "teams": ["RR", "Delhi Capitals"]
    },
    {
        "tweet": "Superb innings by Pandey takes Sunrisers Hyderabad to victory over Mumbai Indians. #SunrisersHyderabadvsMumbaiIndians",
        "players": ["Pandey"],
        "teams": ["Sunrisers Hyderabad", "Mumbai Indians"]
    },
    {
        "tweet": "Outstanding performance by Narine as Kolkata Knight Riders secure a win over CSK. #KolkataKnightRidersvsChennaiSuperKings",
        "players": ["Narine"],
        "teams": ["Kolkata Knight Riders", "CSK"]
    },
    {
        "tweet": "Clinical chase by Padikkal and Kohli ensures a comfortable victory for RCB over Punjab Kings. #RoyalChallengersBangalorevsPunjabKings",
        "players": ["Padikkal", "Kohli"],
        "teams": ["RCB", "Punjab Kings"]
    },
    {
        "tweet": "Brilliant performance by Archer as Rajasthan Royals thrash DC by a huge margin. #RajasthanRoyalsvsDelhiCapitals",
        "players": ["Archer"],
        "teams": ["Rajasthan Royals", "DC"]
    },
    {
        "tweet": "Sensational bowling by Rashid Khan helps SRH defend a low total against KKR. #SunrisersHyderabadvsKolkataKnightRiders",
        "players": ["Rashid Khan"],
        "teams": ["SRH", "KKR"]
    },
    {
        "tweet": "Magnificent century by Rahul goes in vain as PBKS fall short against MI. #PunjabKingsvsMumbaiIndians",
        "players": ["Rahul"],
        "teams": ["PBKS", "MI"]
    },
    {
        "tweet": "Vintage Gayle storm helps Punjab Kings chase down a massive target against Delhi Capitals. #PunjabKingsvsDelhiCapitals",
        "players": ["Gayle"],
        "teams": ["Punjab Kings", "Delhi Capitals"]
    },
    {
        "tweet": "Bravo's stunning catch secures victory for Chennai Super Kings against RR. What a game! #ChennaiSuperKingsvsRajasthanRoyals",
        "players": ["Bravo"],
        "teams": ["Chennai Super Kings", "RR"]
    },
    {
        "tweet": "Dominant performance by Rashid Khan as Sunrisers Hyderabad thrash Royal Challengers Bangalore by a huge margin. #SunrisersHyderabadvsRoyalChallengersBangalore",
        "players": ["Rashid Khan"],
        "teams": ["Sunrisers Hyderabad", "Royal Challengers Bangalore"]
    },
    {
        "tweet": "Clinical chase by Samson and Buttler ensures a comfortable victory for Rajasthan Royals over Kolkata Knight Riders. #RajasthanRoyalsvsKolkataKnightRiders",
        "players": ["Samson", "Buttler"],
        "teams": ["Rajasthan Royals", "Kolkata Knight Riders"]
    }
]

In [15]:
file_path = "../data/clean_tweets.csv"

In [16]:
# NEW
def extract_entities_from_gold(gold_standard):
    """
    Extracts entities from the gold standard entries based on start and end positions.
    Returns a list of lists of (entity, label) tuples.
    """
    gold_entities = []
    for entry in gold_standard:
        text = entry["tweet"]
        entities = []
        for entity in entry["players"]:
            entities.append((entity, 'Player'))
        for team in entry["teams"]:
            entities.append((team, 'Team'))
        gold_entities.append(entities)
    return gold_entities

In [17]:
# NEW
def evaluate_metrics(model_output, gold_standard):
    TP, FP, FN = 0, 0, 0

    gold_entities = extract_entities_from_gold(gold_standard)

    # Assume each entry in model_output corresponds to an entry in gold_entities
    for model_entities, gold_entities_for_text in zip(model_output, gold_entities):
        model_set = set(model_entities)
        gold_set = set(gold_entities_for_text)

        TP += len(model_set & gold_set)
        FP += len(model_set - gold_set)
        FN += len(gold_set - model_set)

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    accuracy = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0

    return precision, recall, f1_score, accuracy

In [18]:
raw_text = list(map(lambda x: {'text': x['tweet']}, annotated_data))
data = pd.DataFrame(raw_text)
data.head()

,text
0,What a thrilling match! Rahul's magnificent ce...
1,Superb performance by Dhoni as he steers Chenn...
2,Incredible bowling display by Bumrah helps Mum...
3,Great partnership between Kohli and de Villier...
4,Russell's explosive batting takes KKR to victo...


# Evaluation for Rules Based

In [19]:
rules_info_ext(file_path)

(None,
 {"('Rajasthan Royals', 'Team')": 12,
  "('Sunrisers Hyderabad', 'Team')": 11,
  "('Royal Challengers Bangalore', 'Team')": 11,
  "('RR', 'Team')": 11,
  "('Punjab Kings', 'Team')": 10,
  "('Mumbai Indians', 'Team')": 10,
  "('Kolkata Knight Riders', 'Team')": 10,
  "('MI', 'Team')": 10,
  "('SRH', 'Team')": 10,
  "('RCB', 'Team')": 10,
  "('Delhi Capitals', 'Team')": 9,
  "('Chennai Super Kings', 'Team')": 9,
  "('KKR', 'Team')": 9,
  "('CSK', 'Team')": 8,
  "('DC', 'Team')": 8,
  "('PBKS', 'Team')": 8,
  "('Kohli', 'Player')": 5,
  "('Bravo', 'Player')": 5,
  "('Pandya', 'Player')": 5,
  "('Rashid Khan', 'Player')": 5,
  "('Archer', 'Player')": 5,
  "('Rahul', 'Player')": 4,
  "('Villiers', 'Player')": 4,
  "('Warner', 'Player')": 4,
  "('Vintage Gayle', 'Player')": 4,
  "('Superb', 'Player')": 3,
  "('Stokes', 'Player')": 3,
  "('Dhawan', 'Player')": 3,
  "('Samson', 'Player')": 3,
  "('Buttler', 'Player')": 3,
  "('Bumrah', 'Player')": 2,
  "('Rabada', 'Player')": 2,
  "('Fa

In [20]:
pred_rules = run_rules(data, eval=True)
precision_rules, recall_rules, f1_rules, accuracy_rules = evaluate_metrics(pred_rules, annotated_data)
print(f"Precision: {precision_rules}, Recall: {recall_rules}, F1-Score: {f1_rules}, Accuracy: {accuracy_rules}")

Precision: 0.9322709163346613, Recall: 0.9590163934426229, F1-Score: 0.9454545454545454, Accuracy: 0.896551724137931


# Evaluation for SpaCy (CNN & RNN)

In [29]:
spacy_info_ext(file_path)

(None,
 {"('Kolkata Knight Riders', 'Player')": 10,
  "('RCB', 'Team')": 10,
  "('Delhi Capitals', 'Team')": 9,
  "('KKR', 'Team')": 9,
  "('Rajasthan Royals', 'Team')": 8,
  "('CSK', 'Team')": 8,
  "('SRH', 'Team')": 6,
  "('Royal Challengers Bangalore', 'Player')": 6,
  "('Chennai Super Kings', 'Team')": 5,
  "('Royal Challengers Bangalore', 'Team')": 5,
  "('Bravo', 'Team')": 5,
  "('Pandya', 'Player')": 5,
  "('Rashid Khan', 'Player')": 5,
  "('Kohli', 'Player')": 4,
  "('Warner', 'Team')": 4,
  "('Samson', 'Team')": 3,
  "('Bumrah', 'Player')": 2,
  "('Sunrisers Hyderabad', 'Player')": 2,
  "('Dhawan', 'Player')": 2,
  "('Archer as Rajasthan Royals', 'Team')": 2,
  "('Chahal', 'Team')": 2,
  "('Ferguson', 'Team')": 2,
  "('Narine', 'Team')": 2,
  "('Mumbai Indians', 'Team')": 2,
  "('Chennai Super Kings', 'Player')": 2,
  "('Superb', 'Player')": 2,
  "('Pandey', 'Team')": 2,
  "('Watson', 'Player')": 2,
  "('Morris', 'Player')": 2,
  "('Buttler', 'Team')": 2,
  "('Rahul', 'Player'

In [30]:
pred_spacy = run_spacy(data, eval=True)
precision_spacy, recall_spacy, f1_spacy, accuracy_spacy = evaluate_metrics(pred_spacy, annotated_data)
print(f"Precision: {precision_spacy}, Recall: {recall_spacy}, F1-Score: {f1_spacy}, Accuracy: {accuracy_spacy}")

Precision: 0.62, Recall: 0.38114754098360654, F1-Score: 0.47208121827411165, Accuracy: 0.3089700996677741


# Evaluation for Stanford NER (CRF)

In [23]:
stanford_info_ext(file_path, display=False)

(None,
 {'Team': {'Punjab Kings': 8,
   'Mumbai Indians': 8,
   'Royal Challengers Bangalore': 7,
   'Rajasthan Royals': 7,
   'SRH': 6,
   'Chennai Super Kings': 5,
   'KKR': 5,
   'CSK': 5,
   'Sunrisers Hyderabad': 3,
   'Chennai Super Kings.': 2,
   'Royal Challengers Bangalore.': 2,
   'Warner': 2,
   'Sunrisers Hyderabad.': 1,
   'Delhi Capitals': 1,
   'Mumbai Indians.': 1,
   'de Villiers': 1,
   'Unstoppable Warner': 1},
  'Player': {'Rashid Khan': 5,
   'Archer': 5,
   'Kohli': 4,
   'Pandya': 4,
   'Rahul': 3,
   'Bumrah': 2,
   'Vintage Gayle': 2,
   'Rabada': 2,
   'de Villiers': 2,
   'Chahal': 2,
   'Gayle': 2,
   'Pandey': 2,
   'Padikkal': 2,
   'Morris': 2,
   'Samson': 2,
   'Dhoni': 1,
   'Kuldeep': 1,
   'Hardik Pandya': 1,
   'Stokes': 1,
   'Smith': 1,
   'Williamson': 1,
   'Bairstow': 1,
   'Iyer': 1,
   'Vintage Dhoni': 1,
   'Bravo': 1,
   'Ferguson': 1,
   'Mayank': 1,
   'Chahar': 1,
   'Nortje': 1,
   'Dhawan': 1,
   'Watson': 1,
   'Narine': 1,
   'Buttle

In [24]:
pred_stanford = run_stanford(data, eval=True, display=False)
precision_stanford, recall_stanford, f1_stanford, accuracy_stanford = evaluate_metrics(pred_stanford, annotated_data)
print(f"Precision: {precision_stanford}, Recall: {recall_stanford}, F1-Score: {f1_stanford}, Accuracy: {accuracy_stanford}")

Precision: 0.8870967741935484, Recall: 0.45081967213114754, F1-Score: 0.5978260869565217, Accuracy: 0.4263565891472868
